# N-Queens

Cost function:


$\begin{equation}
f(x) =  -\sum_i^n \sum_j^n x_{i,j} + \lambda_1(\sum_{i}^n \sum_{j}^n x_{i,j} - N)^2 + \lambda_2\sum_{i_1}^n \sum_{i_2}^n \sum_{i_3}^n \sum_{i_4}^n J_{i_1,i_2,i_3,i_4}x_{i_1,i_2}x_{i_3,i_4}
\end{equation}$

Where $x_{ij}$ = 1 if cell (i,j) contains a queen, 0 otherwise.

The second term, weighted by $\lambda_1$, comes from the constraint that there should be exactly N queens.

The last term, weighted by $\lambda_2$, comes from the constraint that no two queens should attack eachother.
$J_{i_1,i_2,i_3,i_4}$ = 1 if cells $(i_1,i_2)$ y $(i_3, i_4)$ are connected, 0 otherwise.

Expansion of the second term yields linear terms ($x_{ij}$ and $x_{ij}²$) with coefficients $1-2N$, and quadratic terms ($x_{ij}x_{kl}$) with coefficients $2$ (plus a constant term, $N^2$, that can be ommited).

Inclusion of the constraint that there should be exactly N queens (second term) makes much more probable to obtain solutions with exactly N queens, specially for large N. This allows to reduce the number of samples considerably (by orders of magnitude).

In [629]:
def q_dict_from_qubo(Q):
    q_dict = {}
    for i in Q:
        if len(i) == 1:
            q_dict[(i[0],i[0])] = Q[i]
        else:
            q_dict[i] = Q[i]
    return q_dict

In [630]:
def print_solution(solution):
    n_queens = 0
    for i in range(size):
        for j in range(size):
            if solution[f"x_{i}_{j}"] == 0:
                print("O", end = " ")
            else:
                n_queens += 1
                print("X", end = " ")
        print()
    print(f"num. queens: {n_queens} / {size}")

In [631]:
import qubovert

size = 8 # NxN board, up to N queens.

# Penalties for constraints.
lagrange_attack = 2*size # ** 2
lagrange_exact_n = size

# Samples and problem label.
num_reads_simu = 10
num_reads_real = 100
problem_label_simu = f"N-queens size {size}, {num_reads_simu} num_reads"
problem_label_real = f"N-queens size {size}, {num_reads_real} num_reads"

# Create vars.
Q = qubovert.QUBO()
for i in range(size):
    for j in range(size):
        Q.create_var(f"x_{i}_{j}")


In [632]:
# Build cost function: 1/2.

#  - Cost function: Add as many queens as possible: lin_term = -1
#  - Penalty: Add exactly N queens: lin_term = lagrange*(1-2N)
lin_term = -1
lin_term_penalty = lagrange_exact_n * (1 - 2*size)
for i in range(size):
    for j in range(size):
        idx = (f"x_{i}_{j}",)
        Q[idx] = lin_term + lin_term_penalty


In [633]:
# Build cost function: 2/2.
#  - Penalty: Add exactly N queens: cross_term = +2
#  - Penalize any two queens attacking eachother.
penalty_num = lagrange_exact_n * 2
penalty_attack =  lagrange_attack * 1
for i1 in range(size):
    for i2 in range(size):
        for i3 in range(size):
            for i4 in range(size):
                if not (i1 == i3 and i2 == i4):
                    idx = (f"x_{i1}_{i2}", f"x_{i3}_{i4}")
                    term = 0
                    # From the requirement "Add exactly N queens".
                    term += penalty_num
                    if i1 == i3 or i2 == i4 or i1 - i3 == i2 - i4 or i1 - i3 == i4 - i2:
                        term += penalty_attack
                    Q[idx] = term

In [634]:
Q[('x_0_0', 'x_0_1')], Q[('x_0_1', 'x_0_0')]

(32, 32)

In [635]:
len(Q) # 326 for size=6

2080

In [636]:
q_dict = q_dict_from_qubo(Q)
q_dict.get(('x_0_0', 'x_0_1')), q_dict.get(('x_0_1', 'x_0_0')), # 36, None for size=6

(32, None)

In [637]:
from neal import SimulatedAnnealingSampler

sampler_simu = SimulatedAnnealingSampler()

In [638]:
%%time
sampleset_simu = sampler_simu.sample_qubo(q_dict, label=problem_label_simu, num_reads=num_reads_simu)

CPU times: user 35.7 ms, sys: 0 ns, total: 35.7 ms
Wall time: 34 ms


In [639]:
solution_simu = sampleset_simu.first.sample
print_solution(solution_simu)

O O X O O O O O 
O O O O O X O O 
O O O O O O O X 
X O O O O O O O 
O O O O X O O O 
O O O O O O X O 
O X O O O O O O 
O O O X O O O O 
num. queens: 8 / 8


In [640]:
q_dict = q_dict_from_qubo(Q)
q_dict.get(('x_0_0', 'x_0_1')), q_dict.get(('x_0_1', 'x_0_0')), # 36, None for size=6

(32, None)

In [641]:
from dwave.system import DWaveSampler, EmbeddingComposite

sampler_real = EmbeddingComposite(DWaveSampler())

In [642]:
%%time
sampleset_real = sampler_real.sample_qubo(q_dict, label=problem_label_real, num_reads=num_reads_real)

CPU times: user 21.1 s, sys: 157 ms, total: 21.3 s
Wall time: 20.9 s


In [643]:
solution_real = sampleset_real.first.sample
print_solution(solution_real)

O O O O X X O O 
O X O O O O O O 
O O O O O O O O 
O O O O O O X X 
O O X O O O O O 
O O O O O O O O 
O O O O O O O O 
X O O O O O O O 
num. queens: 7 / 8
